In [7]:
import os
import sys
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from collections import Counter
from multiprocessing import Pool

In [56]:
# lnc_bed = "/data3/xs/tissue_m6a/2018.1/GRCh38_segment/mRNA.bed"
lnc_bed = "/data3/xs/tissue_m6a/2018.1/GRCh38_segment/intergenic_lincRNA.bed"
total_gene_dir = "/data5/galaxy/project/promoter_TF_enrich/data/total_gene/gene_bed"
m6a_gene_dir = "/data5/galaxy/project/promoter_TF_enrich/data/m6a_gene/gene_bed"
#
expre_file = "/data5/galaxy/project/expression/stringtie/Total-RPKM.txt"
#
as_dir = "/data5/xlj/new_tissue_m6A/7_10_2018_Figure1/AS/"

In [57]:
total_gene_list = glob.glob("%s/*.bed" % total_gene_dir)
m6a_gene_list = glob.glob("%s/*.bed" % m6a_gene_dir)
df_lnc = read_bed(lnc_bed)
df_exp = pd.read_table(expre_file, sep="\t")
df_exp.columns = ["gene", "brain", "heart", "kidney", "liver", "placenta", "lung", "muscle", "stomach"]
df_exp["name"] = df_exp["gene"].str.split(".").str[0]
# df_lnc.head()
tissue_list, total_list, m6a_list = [], [], []
for gene_bed in total_gene_list:
    tissue = os.path.basename(gene_bed).split(".bed")[0]
    print(tissue)
    df_m6a, df_unm6a = process_single_tissue(gene_bed, df_lnc)
#     print(len(df_m6a), len(df_unm6a), len(df_lnc))
#     as_file = os.path.join("%s/%s" % (as_dir, tissue.title()), "AS_Total")
#     as_num_dict = count_each_as(as_file)
#     as_propor_1 = len(df_m6a[df_m6a["name"].isin(as_num_dict.keys())]) / len(df_m6a)
#     as_propor_2 = len(df_unm6a[df_unm6a["name"].isin(as_num_dict.keys())]) / len(df_unm6a)
#     print("%s\n%f\t%f" % (tissue, as_propor_1, as_propor_2))
    
#     result_m6a = get_as_num("m6a lncRNA", df_m6a, as_num_dict)
#     result_unm6a = get_as_num("unm6a lncRNA", df_unm6a, as_num_dict)
#     write_file(tissue, (result_m6a + result_unm6a))
    
#     compare_expre(tissue, df_exp, df_m6a, df_unm6a)
#     result_list = process_single_tissue(gene_bed, df_lnc)
#     tissue_list.append(result_list[0])
#     total_list.append(result_list[1])
#     m6a_list.append(result_list[2])
#     pro_list = 100 * (np.array(m6a_list) / np.array(total_list))
# with open("/data5/galaxy/project/lncRNA_analysis/m6a_proportion/proportion.txt", 'w') as fw:
#     fw.write("tissue name\t%s\n" % ("\t".join(tissue_list)))
#     fw.write("total lncRNA number\t%s\n" % ("\t".join([str(x) for x in total_list])))
#     fw.write("m6a lncRNA number\t%s\n" % ("\t".join([str(x) for x in m6a_list])))
#     fw.write("m6a lncRNA proportion\t%s\n" % ("\t".join([str(x) for x in pro_list])))

liver
2301 485 1816
stomach
1921 416 1505
placenta
2150 690 1460
lung
2049 471 1578
heart
2826 501 2325
brain
3989 836 3153
muscle
1958 655 1303
kidney
2651 967 1684


In [53]:
def process_single_tissue(gene_bed, df_lnc):
    tissue = os.path.basename(gene_bed)
#     print(tissue.split(".bed")[0])
    m6a_bed = os.path.join(m6a_gene_dir, tissue)
    df_gene, df_m6a = read_bed(gene_bed), read_bed(m6a_bed)
#     print(len(df_gene), len(df_m6a))
#     print(df_gene.head())
#     print(len(df_lnc))
    df_one = df_gene.merge(df_lnc, on="name", how="right").dropna()
    total_names = list(set(df_one["name"].tolist()))
#     print(len(df_one))
#     print(df_lnc.head())
    df_two = df_one.merge(df_m6a, on="name", how="left").dropna()
    m6a_names = list(set(df_two["name"].tolist()))
#     print(len(df_two))
    unm6a_names = list(set([x for x in df_one["name"].tolist() if x not in df_two["name"].tolist()]))
    df_three = df_one[df_one["name"].isin(unm6a_names)].dropna()
    remain_names = list(set(df_three["name"].tolist()))
    print(len(total_names), len(m6a_names), len(remain_names))
#     print(len(df_two), len(df_three))
#     print(len(df_two))
#     result_list = [tissue.split(".bed")[0], len(df_one), len(df_two)]
#     return result_list
    return df_two, df_three

In [36]:
def read_bed(in_bed):
    df = pd.read_table(in_bed, sep="\s+", header=None, names=["chr", "start", "end", "name", "s", "strand"])
#     print(df.head())
    df["name"] = df["name"].str.split(".").str[0]
    return df

In [57]:
def compare_expre(tissue, df_exp, df_m6a, df_unm6a):
    df_sub = df_exp[["name", tissue]]
    df_m6a_exp = df_sub.merge(df_m6a, on="name", how="left").dropna()[["name", tissue]]
    df_m6a_exp["type"] = "m6a"
    df_unm6a_exp = df_sub.merge(df_unm6a, on="name", how="left").dropna()[["name", tissue]]
    df_unm6a_exp["type"] = "unm6a"
    print(len(df_m6a_exp), len(df_unm6a_exp))
    print(df_m6a_exp[tissue].median(), df_unm6a_exp[tissue].median())
    print(stats.mannwhitneyu(df_m6a_exp[tissue], df_unm6a_exp[tissue]))
    df = pd.concat([df_m6a_exp, df_unm6a_exp])
    df = df[["name", "type", tissue]]
    df.to_csv("/data5/galaxy/project/lncRNA_analysis/m6a_expression/%s.txt" % tissue, sep="\t", index=False)

In [5]:
def count_each_as(as_file):
    with open(as_file, 'r') as f:
        title = f.readline()
        names = [line.split()[1].split(".")[0].strip('"') for line in f.readlines()]
        num_dict = Counter(names)
    return num_dict

def get_as_num(lnc_type, df, as_num_dict):
    df_in = df[df["name"].isin(list(as_num_dict.keys()))]
    result_list = []
    for index, x in df_in.iterrows():
        number = as_num_dict[x["name"]]
        result_list.append("%s\t%s\t%s\n" % (lnc_type, x["name"], str(number)))
    return result_list

def write_file(tissue, result_list):
    result_file = "/data5/galaxy/project/lncRNA_analysis/as_number/%s.txt" % tissue
    with open(result_file, 'w') as fw:
        fw.writelines(result_list)